In [1]:
import pandas as pd
import numpy as np
import py_stringmatching as sm
import py_stringsimjoin as ssj
from pandas_profiling import ProfileReport
import re
import nltk
from datetime import datetime
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)

In [4]:
ban = pd.read_csv('../../datasets/books/ban.csv',dtype='str',sep=';')

In [5]:
bx = pd.read_csv('../../datasets/books/bx.csv',dtype='str',sep=';')

In [6]:
ban.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17629 entries, 0 to 17628
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ban_id                          17629 non-null  object
 1   ban_isbn                        17629 non-null  object
 2   ban_title                       17628 non-null  object
 3   ban_author                      16277 non-null  object
 4   ban_binding                     2961 non-null   object
 5   ban_pubdate                     8822 non-null   object
 6   ban_pages                       17629 non-null  object
 7   ban_publisher                   17589 non-null  object
 8   ban_title_no_par                17628 non-null  object
 9   ban_title_no_par_no_stop_words  17615 non-null  object
 10  ban_firstauthor                 16277 non-null  object
 11  ban_firsttwoauthors             16277 non-null  object
dtypes: object(12)
memory usage: 1.6+ MB


In [7]:
bx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270948 entries, 0 to 270947
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   bx_id                          270948 non-null  object
 1   bx_isbn                        270948 non-null  object
 2   bx_title                       270948 non-null  object
 3   bx_firstauthor                 270947 non-null  object
 4   bx_pubdate                     270948 non-null  object
 5   bx_pages                       270948 non-null  object
 6   bx_publisher                   270946 non-null  object
 7   bx_binding                     4171 non-null    object
 8   bx_title_no_par                270946 non-null  object
 9   bx_title_no_par_no_stop_words  270658 non-null  object
dtypes: object(10)
memory usage: 20.7+ MB


In [8]:
ban.fillna('',inplace=True)
bx.fillna('',inplace=True)

In [9]:
matches_ban_bx = pd.merge(ban,bx,left_on='ban_isbn',right_on='bx_isbn')
print('True matches across ban and bx: {}'.format(matches_ban_bx.shape[0]))

True matches across ban and bx: 699


In [10]:
ban.columns

Index(['ban_id', 'ban_isbn', 'ban_title', 'ban_author', 'ban_binding', 'ban_pubdate', 'ban_pages', 'ban_publisher', 'ban_title_no_par', 'ban_title_no_par_no_stop_words', 'ban_firstauthor', 'ban_firsttwoauthors'], dtype='object')

In [11]:
bx.columns

Index(['bx_id', 'bx_isbn', 'bx_title', 'bx_firstauthor', 'bx_pubdate', 'bx_pages', 'bx_publisher', 'bx_binding', 'bx_title_no_par', 'bx_title_no_par_no_stop_words'], dtype='object')

### Join on author on Jaccard score of at least 0.3 with 3-gram tokenization

In [10]:
q3 = sm.QgramTokenizer(qval=3,return_set=True)

In [13]:
ban_bx_pairs_a_q3_jac_03 = ssj.jaccard_join(ban, bx, 'ban_id', 'bx_id', 'ban_author_lower', 'bx_author_lower', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_author','ban_author_lower','ban_firstauthor','ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['bx_isbn','bx_title','bx_title_lower','bx_title_no_par','bx_author','bx_author_lower','bx_binding','bx_pubdate','bx_pages','bx_publisher'],
                                     n_jobs=-3)

### Join on first author based (ban) and author (bx) on Jaccard score of at least 0.3 with 3-gram tokenization

In [12]:
ban_bx_pairs_fa_q3_jac_03 = ssj.jaccard_join(ban, bx, 'ban_id', 'bx_id', 'ban_firstauthor', 'bx_author_lower', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_author','ban_author_lower','ban_firstauthor','ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['bx_isbn','bx_title','bx_title_lower','bx_title_no_par','bx_author','bx_author_lower','bx_binding','bx_pubdate','bx_pages','bx_publisher'],
                                     n_jobs=-3)

### Join on title (with parentheses) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [14]:
ban_bx_pairs_tit_q3_jac_03 = ssj.jaccard_join(ban, bx, 'ban_id', 'bx_id', 'ban_title', 'bx_title', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_author','ban_author_lower','ban_firstauthor','ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['bx_isbn','bx_title','bx_title_lower','bx_title_no_par','bx_author','bx_author_lower','bx_binding','bx_pubdate','bx_pages','bx_publisher'],
                                     n_jobs=-3)

### Join on title (without parentheses) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [15]:
ban_bx_pairs_titnp_q3_jac_03 = ssj.jaccard_join(ban, bx, 'ban_id', 'bx_id', 'ban_title_no_par', 'bx_title_no_par', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_author','ban_author_lower','ban_firstauthor','ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['bx_isbn','bx_title','bx_title_lower','bx_title_no_par','bx_author','bx_author_lower','bx_binding','bx_pubdate','bx_pages','bx_publisher'],
                                     n_jobs=-3)

### Join on title (w/out parentheses) and stop words removed based on Jaccard score of at least 0.3 with 3-gram tokenization

In [17]:
ban_bx_pairs_titnp_nosw_q3_jac_03 = ssj.jaccard_join(ban, bx, 'ban_id', 'bx_id', 'ban_title_no_par_no_stop_words', 'bx_title_no_par_no_stop_words', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_title_no_par_no_stop_words','ban_author','ban_author_lower','ban_firstauthor', 'ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['bx_isbn','bx_title','bx_title_lower','bx_title_no_par','bx_title_no_par_no_stop_words','bx_author', 'bx_author_lower','bx_binding','bx_pubdate','bx_pages','bx_publisher'],
                                     n_jobs=-3)

### Evaluate Blocking

In [18]:
print('True matches across Ban and bxdery: {}'.format(matches_ban_bx.shape[0]))

True matches across Ban and bxdery: 699


In [19]:
def assignTrueLabels(df_blockedpairs, idcolumn1, idcolumn2):
    return df_blockedpairs.apply(lambda row: 1 if (row[idcolumn1]==row[idcolumn2]) else 0, axis=1)

In [20]:
def calcRedRatio(df1,df2,df_blockedpairs):
    cp = (df1.shape[0]*df2.shape[0])/2
    rr = (1-(df_blockedpairs.shape[0]/cp))*100
    return round(rr,2)

In [21]:
def calcPairCompISBN(df_matches,df_blockedpairs,true_label_col):
    pc = (df_blockedpairs[df_blockedpairs[true_label_col]==1].shape[0]/df_matches.shape[0])*100
    return round(pc,2)

In [22]:
#Assign labels to the blockedpairs df
ban_bx_pairs_a_q3_jac_03['true_label'] = assignTrueLabels(ban_bx_pairs_a_q3_jac_03,'l_ban_isbn','r_bx_isbn')
ban_bx_pairs_fa_q3_jac_03['true_label'] = assignTrueLabels(ban_bx_pairs_fa_q3_jac_03,'l_ban_isbn','r_bx_isbn')
ban_bx_pairs_tit_q3_jac_03['true_label'] = assignTrueLabels(ban_bx_pairs_tit_q3_jac_03,'l_ban_isbn','r_bx_isbn')
ban_bx_pairs_titnp_q3_jac_03['true_label'] = assignTrueLabels(ban_bx_pairs_titnp_q3_jac_03,'l_ban_isbn','r_bx_isbn')
ban_bx_pairs_titnp_nosw_q3_jac_03['true_label'] = assignTrueLabels(ban_bx_pairs_titnp_nosw_q3_jac_03,'l_ban_isbn','r_bx_isbn')

In [24]:
print('Blocking based on author, Jaccard, 3-gram, Threshold 0.3')
print('Evaluation of ban_bx_pairs_a_q3_jac_03')
print('Number of rows: {:,}'.format(ban_bx_pairs_a_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,bx,ban_bx_pairs_a_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_bx,ban_bx_pairs_a_q3_jac_03,'true_label')))
print()
print('Blocking based on first author, Jaccard, 3-gram, Threshold 0.3')
print('Evaluation of ban_bx_pairs_fa_q3_jac_03')
print('Number of rows: {:,}'.format(ban_bx_pairs_fa_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,bx,ban_bx_pairs_fa_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_bx,ban_bx_pairs_fa_q3_jac_03,'true_label')))

print('\n')
print('Blocking based on title')

print('Evaluation of ban_bx_pairs_tit_q3_jac_03 (Title, Jaccard, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(ban_bx_pairs_tit_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,bx,ban_bx_pairs_tit_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_bx,ban_bx_pairs_tit_q3_jac_03,'true_label')))
print()
print('Evaluation of ban_bx_pairs_titnp_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(ban_bx_pairs_titnp_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,bx,ban_bx_pairs_titnp_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_bx,ban_bx_pairs_titnp_q3_jac_03,'true_label')))
print()
print('Evaluation of ban_bx_pairs_titnp_nosw_q3_jac_03 (Title excluded text in parenthesis and stop words removed, Jaccard, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(ban_bx_pairs_titnp_nosw_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,bx,ban_bx_pairs_titnp_nosw_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_bx,ban_bx_pairs_titnp_nosw_q3_jac_03,'true_label')))

Blocking based on author, Jaccard, 3-gram, Threshold 0.3
Evaluation of ban_bx_pairs_a_q3_jac_03
Number of rows: 3,475,565
Reduction Ratio: 99.85%
Pair Completeness: 92.27%
Blocking based on first author, Jaccard, 3-gram, Threshold 0.3
Evaluation of ban_bx_pairs_fa_q3_jac_03
Number of rows: 3,615,536
Reduction Ratio: 99.85%
Pair Completeness: 92.27%


Blocking based on title
Evaluation of ban_bx_pairs_tit_q3_jac_03 (Title, Jaccard, 3-gram, Threshold 0.3)
Number of rows: 204,456
Reduction Ratio: 99.99%
Pair Completeness: 91.85%


Evaluation of ban_bx_pairs_titnp_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)
Number of rows: 45,545
Reduction Ratio: 100.0%
Pair Completeness: 67.53%


Evaluation of ban_bx_pairs_titnp_nosw_q3_jac_03 (Title excluded text in parenthesis and stop words removed, Jaccard, 3-gram, Threshold 0.3)
Number of rows: 391,304
Reduction Ratio: 99.98%
Pair Completeness: 89.41%


In [26]:
ban_bx_pairs_tit_q3_jac_03.to_csv('filtered_correspondences/ban_bx_pairs_tit_q3_jac_03.csv',index=False)

In [101]:
matches_ban_bx.to_csv('matching_pairs/matches_ban_bx.csv',index=False)

-- Blocking based on Levenshtein performs significantly worse than Blocking on Jaccard distance with 3-gram tokens and blocking on title also performs worse compared to authors. Blocking on the firsttwoauthors performs best. Same results as with ban and wordery